In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

In [ ]:
!unzip './drive/My Drive/audio-video.zip'

Archive:  ./drive/My Drive/audio-video.zip
   creating: audio-video/
 extracting: audio-video/Video_Speech_Actor_01.zip  
 extracting: audio-video/Video_Speech_Actor_02.zip  
 extracting: audio-video/Video_Speech_Actor_03.zip  
 extracting: audio-video/Video_Speech_Actor_04.zip  
 extracting: audio-video/Video_Speech_Actor_05.zip  
 extracting: audio-video/Video_Speech_Actor_06.zip  
 extracting: audio-video/Video_Speech_Actor_07.zip  
 extracting: audio-video/Video_Speech_Actor_08.zip  
 extracting: audio-video/Video_Speech_Actor_09.zip  
 extracting: audio-video/Video_Speech_Actor_10.zip  
 extracting: audio-video/Video_Speech_Actor_11.zip  
 extracting: audio-video/Video_Speech_Actor_12.zip  
 extracting: audio-video/Video_Speech_Actor_13.zip  
 extracting: audio-video/Video_Speech_Actor_14.zip  
 extracting: audio-video/Video_Speech_Actor_15.zip  
 extracting: audio-video/Video_Speech_Actor_16.zip  
 extracting: audio-video/Video_Speech_Actor_17.zip  
 extracting: audio-video/Video

In [ ]:
#!unzip 'audio-video/Video_Speech_Actor_01.zip'

In [ ]:
import os, shutil, numpy as np, cv2,zipfile
import subprocess
from google.colab import files
from IPython.display import Audio
import librosa
import matplotlib.pyplot as plt 
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.optimizers import Adam,SGD
import tensorflow

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=True)

<h2>Loading dataset</h2>
I start with loading dataset. Dataset contain auido-video (label=1), only video (label=2), only audio (label=3). </br>
<b>The target of classification:</b></br>
We have 8 labels.</br>
01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised.</br>

<b>Steps:</b> I extract the dataset initially in seprate arrays based on modality. </br>
<b>Process of extracting video frames:</b>
I used open CV for this purpose. Read the video frame by frame, and extract feature for the frame using pretrained resnet-50 model. We store these features.</br>
<b>Process of extracting audio:</b>
I used librosa library for this purpose. <b>MFCC</b> (Mel Freequency capstral coefficients). These are already proven to perform better on classification tasks.</b>
Complete audio which is of 2,3, or 4 second is considered for MFCC feature.</b>

<b>If label = audio-video.</b></br>



In [ ]:
for i in range(1,9):
    os.mkdir(str(i))
    #shutil.rmtree(str(i))

In [ ]:
#shutil.rmtree('zipfiles')

In [ ]:
ls

1/  2/  3/  4/  5/  6/  7/  8/  audio-video/  drive/  sample_data/  zipfiles/


In [ ]:
arr1,arr2,arr3=[],[],[]
label1,label2,label3=[],[],[]

for act in range(1,21):
    #'audio-video/Video_Speech_'+'Actor_'+str(act).zfill(2)+".zip"
    
    with zipfile.ZipFile('audio-video/Video_Speech_'+'Actor_'+str(act).zfill(2)+".zip", 'r') as zip_ref:
        zip_ref.extractall('zipfiles')
    
    l=os.listdir('zipfiles/Actor_'+str(act).zfill(2)+'/')
    for ff in range(len(l)):
        #print('Actor_'+str(act).zfill(2)+'/'+l[ff])
        
        doc_name=l[ff]
        n=doc_name.split('.mp4')
        w=int(n[0].split('-')[0])#01 = full-AV, 02 = video-only, 03 = audio-only
        lab=int(n[0].split('-')[2])#01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised


        if (w==1):#audio video both
            # nn=str(vid[0])
            # vid_name="/normal_final/"+nn
            arr=[]
            video = cv2.VideoCapture('zipfiles/Actor_'+str(act).zfill(2)+'/'+l[ff])
            while(True): 
                ret, frame = video.read() 
                if ret == True:  
                    #input=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
                    frame=cv2.resize(frame,(224,224))
                    feat=frame.reshape(1,224,224,3)
                    arr.append(base_model.predict(feat))
                else:
                    print("done",'zipfiles/Actor_'+str(act).zfill(2)+'/'+l[ff])

                    video.release()
                    cv2.destroyAllWindows()
                    break

            subprocess.call(['ffmpeg', '-i', 'zipfiles/Actor_'+str(act).zfill(2)+'/'+l[ff], str(lab)+'/'+'Actor_'+str(act).zfill(2)+'_'+n[0]+'.mp3'])
            x,sr=librosa.load('1.mp3')
            x2=librosa.to_mono(x)
            mf=librosa.feature.mfcc(x2, sr=22050, n_mfcc=44)
            os.remove('1.mp3') #remove a file
            arr1.append([arr,mf])
            label1.append(lab)
        elif (w==2):
            only video
            arr=[]
            video = cv2.VideoCapture('zipfiles/Actor_'+str(act).zfill(2)+'/'+l[ff])
            while(True): 
                ret, frame = video.read() 
                if ret == True:  
                    #input=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
                    frame=cv2.resize(frame,(224,224))
                    feat=frame.reshape(1,224,224,3)
                    arr.append(base_model.predict(feat))
                else:
                    print("done",'zipfiles/Actor_'+str(act).zfill(2)+'/'+l[ff])
                    video.release()
                    cv2.destroyAllWindows()
                    break
            arr2.append(arr)
            label2.append(lab)
        else:
            only audio
            mf=0
            subprocess.call(['ffmpeg', '-i', 'zipfiles/Actor_'+str(act).zfill(2)+'/'+l[ff], '1.mp3'])
            x,sr=librosa.load('1.mp3')
            x2=librosa.to_mono(x)
            mf=librosa.feature.mfcc(x2, sr=22050, n_mfcc=44)
            os.remove('1.mp3') #remove a file
            arr3.append(mf)
            label3.append(lab)

In [ ]:
ls

1/  2/  3/  4/  5/  6/  7/  8/  audio-video/  drive/  sample_data/  zipfiles/


In [ ]:
n

In [ ]:
for i in range(1,9):
    shutil.copytree(str(i),'./drive/My Drive/'+str(i))

In [ ]:
def feature6(audio):
  #librosa.feature.chroma_stft(audio,sr=22050),librosa.feature.chroma_cqt(audio,sr=22050), 
  return np.concatenate((librosa.feature.chroma_cens(audio,sr=22050),librosa.feature.mfcc(audio,sr=22050,n_mfcc=44) ,librosa.feature.melspectrogram(y=audio,sr=22050),
               librosa.feature.rms(audio), #librosa.feature.rmse(audio), 
               librosa.feature.spectral_centroid(audio,sr=22050),librosa.feature.tempogram(audio,sr=22050),
               librosa.feature.spectral_bandwidth(audio,sr=22050),librosa.feature.spectral_contrast(audio,sr=22050),librosa.feature.spectral_flatness(audio),librosa.feature.spectral_rolloff(audio,sr=22050),
               librosa.feature.poly_features(audio,sr=22050),librosa.feature.tonnetz(audio,sr=22050),librosa.feature.zero_crossing_rate(audio)),axis=0)


In [ ]:
arr1,lab=[],[]
for i in range(1,9):
    #arr=[]
    for j in os.listdir('./drive/My Drive/'+str(i)):
        x,sr=librosa.load('./drive/My Drive/'+str(i)+'/'+j)
        arr1.append(feature6(x))
        lab.append(i)

In [ ]:
from sklearn.utils import shuffle

In [ ]:
# arr,label = shuffle(arr1, lab, random_state=0)
# arr1=0
# lab=0

In [ ]:
arrmax, arrmin=0,10000
for i in arr:#[0].shape
    if arrmax<i.shape[1]:
        arrmax=i.shape[1]
    if arrmin>i.shape[1]:
        arrmin=i.shape[1]
print(arrmin,arrmax)

127 228


In [ ]:
arr2=[]
for i in arr:
    diff=arrmax-i.shape[1]
    arr2.append(np.concatenate((i,np.zeros((i.shape[0],diff))),axis=1))
arr2=np.array(arr2)
arr2.shape

(1200, 589, 228)

In [ ]:
arr2=np.reshape(arr2,(len(arr2),-1,1))

In [ ]:
#arr2=np.array(arr)
# label=np.array(label)
# label=label-1
# label=to_categorical(label)
arr2.shape,label.shape

((1200, 134292), (1200, 8))

In [ ]:
min(arr2.ravel()), max(arr2.ravel())

(-1093.0296630859375, 10895.80078125)

In [ ]:
arr2= (arr2 - )/()

In [ ]:
input = Input(shape=(134292), name='image_input')
#x = Flatten()(base_model(input))
x = Dense(300, activation='tanh')(input)
#x = Dense(100, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(8, activation='softmax')(x)
model = Model(inputs=input, outputs=x)
opt = SGD(learning_rate=1e-3)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(arr2,label, validation_split=0.15,epochs=50)#, batch_size=200)

In [ ]:
import keras

In [ ]:
from tensorflow.keras import Sequential, layers

In [ ]:
model = Sequential()
model.add(layers.Conv1D(256, 5,padding='same',input_shape=(236,40)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling1D(pool_size=(8)))
model.add(layers.Dropout(0.2))

model.add(layers.Conv1D(128, 5,padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling1D(pool_size=(4)))
model.add(layers.Dropout(0.1))

model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Dense(8))
model.add(layers.Activation('softmax'))

opt = SGD(learning_rate=1e-3)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(arr2,label, validation_split=0.15,epochs=50)#, batch_size=200)

In [ ]:
# np.save('./drive/My Drive/audio_video/11',arr1)
# np.save('./drive/My Drive/audio_video/12',arr2)
# np.save('./drive/My Drive/audio_video/13',arr3)
# np.save('./drive/My Drive/audio_video/11l',label1)
# np.save('./drive/My Drive/audio_video/12l',label2)
# np.save('./drive/My Drive/audio_video/13l',label3)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [ ]:
# files.download('11.npy')
# files.download('12.npy')
# files.download('13.npy')
# files.download('11l.npy')
# files.download('12l.npy')
# files.download('13l.npy')

In [ ]:
# from google.colab import drive
# drive.mount("/content/drive/")
# from google.colab import drive
# drive.flush_and_unmount()

Mounted at /content/drive/


Colab provide only 2 cores. 1 run code other run interface. So mutithreading in one file is not efficient.<br>
Hence, I choose to run same code in multiple colab accounts. Actor 1-8 run in this file, Actor 8-15 in second and Actor 15-21 in third. There is no chage in code; Just while loding dataset step, range (1,8) is replaced. However they are stored in one drive, to avoid running again. Also colab GPU is available for limited time.</br>
Below three code blocks load load the feature and label arrays.

In [ ]:
len(arr1),len(label1)

(1200, 1200)

In [ ]:
arr1= np.array(arr1)
label1= np.array(label1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [ ]:
arr1.shape

In [ ]:
audio=[]
for i in arr1:
    audio.append(i[1])
#audio=arr1
label=label1
audio_img=[]
for i in audio:
    fig = plt.Figure(frameon=False)
    fig.set_size_inches(2,1)
    canvas = FigureCanvas(fig)
    #ax = fig.add_subplot(111)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    #mfccs = librosa.feature.mfcc(audio[0], sr=22050)
    ax.imshow(i, aspect='auto')
    fig.savefig('spec1.png')
    audio_img.append(cv2.resize(cv2.imread('spec1.png'),(224,224)))
    os.remove('spec1.png')
    #plt.imshow(plt.imread('spec1.png'))

In [ ]:
audio_img=np.array(audio_img)
label= np.array(label)
label=label-1
label=to_categorical(label)
audio_img.shape,label.shape

((1200, 224, 224, 3), (1200, 8))

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=True)#, input_tensor=Input(shape=(237, 237, 3)))

In [ ]:
# audio_img=audio_img/255.0
audio_img[0]

Accuracy using Just from audio 

In [ ]:
input = Input(shape=(224,224, 3), name='image_input')
for layer in base_model.layers:
    layer.trainable = True
x = Flatten()(base_model(input))
x = Dense(200, activation='relu')(x)
#x = Dense(100, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(8, activation='softmax')(x)
model = Model(inputs=input, outputs=x)
opt = SGD(learning_rate=1e-4)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(audio_img,label, validation_split=0.15,epochs=50)#, batch_size=200)

Epoch 1/50
32/32 [==============================] - 15s 346ms/step - loss: 2.0797 - accuracy: 0.0980 - val_loss: 2.0789 - val_accuracy: 0.1667
Epoch 2/50
32/32 [==============================] - 10s 311ms/step - loss: 2.0795 - accuracy: 0.1020 - val_loss: 2.0796 - val_accuracy: 0.1167
Epoch 3/50
32/32 [==============================] - 10s 312ms/step - loss: 2.0796 - accuracy: 0.0931 - val_loss: 2.0795 - val_accuracy: 0.1278
Epoch 4/50
32/32 [==============================] - 10s 313ms/step - loss: 2.0794 - accuracy: 0.1049 - val_loss: 2.0793 - val_accuracy: 0.0778
Epoch 5/50
32/32 [==============================] - 10s 313ms/step - loss: 2.0795 - accuracy: 0.1275 - val_loss: 2.0792 - val_accuracy: 0.1222
Epoch 6/50
32/32 [==============================] - 10s 314ms/step - loss: 2.0794 - accuracy: 0.1225 - val_loss: 2.0802 - val_accuracy: 0.1500
Epoch 7/50
32/32 [==============================] - 10s 316ms/step - loss: 2.0795 - accuracy: 0.1275 - val_loss: 2.0795 - val_accuracy: 0.1444

In [ ]:
arr1= np.load('./drive/My Drive/audio_video/11.npy', allow_pickle=True)
arr2= np.load('./drive/My Drive/audio_video/12.npy', allow_pickle=True)
arr3= np.load('./drive/My Drive/audio_video/13.npy',allow_pickle=True)
label1=np.load('./drive/My Drive/audio_video/11l.npy',allow_pickle=True)
label2=np.load('./drive/My Drive/audio_video/12l.npy',allow_pickle=True)
label3=np.load('./drive/My Drive/audio_video/13l.npy',allow_pickle=True)

In [ ]:
a21=np.load('./drive/My Drive/audio_video/21.npy',allow_pickle=True)
a22=np.load('./drive/My Drive/audio_video/22.npy',allow_pickle=True)
a23=np.load('./drive/My Drive/audio_video/23.npy',allow_pickle=True)
a21l=np.load('./drive/My Drive/audio_video/21l.npy',allow_pickle=True)
a22l=np.load('./drive/My Drive/audio_video/22l.npy',allow_pickle=True)
a23l=np.load('./drive/My Drive/audio_video/23l.npy',allow_pickle=True)

In [ ]:
a31=np.load('./drive/My Drive/audio_video/31.npy',allow_pickle=True)
a32=np.load('./drive/My Drive/audio_video/32.npy',allow_pickle=True)
a33=np.load('./drive/My Drive/audio_video/33.npy',allow_pickle=True)
a31l=np.load('./drive/My Drive/audio_video/31l.npy',allow_pickle=True)
a32l=np.load('./drive/My Drive/audio_video/32l.npy',allow_pickle=True)
a33l=np.load('./drive/My Drive/audio_video/33l.npy',allow_pickle=True)

MFCC feature will represent audio of video. Video feature are already normalized from Resnet model. In this step, To Normalize the MFCC feature , We find min and max of feature value in training set i.e. (Actor 1 to 21). 

In [ ]:
amin=10000
amax=0
for i in arr1:
    dmin=min(np.ravel(i[1]))
    dmax=max(np.ravel(i[1]))
    if amin>dmin:
        amin=dmin
    if amax<dmax:
        amax=dmax

for i in a21:
    dmin=min(np.ravel(i[1]))
    dmax=max(np.ravel(i[1]))
    if amin>dmin:
        amin=dmin
    if amax<dmax:
        amax=dmax

for i in a31:
    dmin=min(np.ravel(i[1]))
    dmax=max(np.ravel(i[1]))
    if amin>dmin:
        amin=dmin
    if amax<dmax:
        amax=dmax

In [ ]:
amax,amin

(237.81725, -1093.0297)

<h3>Combining the arrays.</h3> </br></br>
a1 is array where each element contain video(frame by frame resnet features) and audio(MFCC) features. </br></br>
a2's each element is just resnet featues as this contains data samples with only video.</br></br>
a3's each element is audio only.

In [ ]:
a1=np.concatenate((arr1,a21,a31),axis=0)
a2=np.concatenate((arr2,a22,a32),axis=0)
a3=np.concatenate((arr3,a23,a33),axis=0)

a1l=np.concatenate((label1,a21l,a31l),axis=0)
a2l=np.concatenate((label2,a22l,a32l),axis=0)
a3l=np.concatenate((label3,a23l,a33l),axis=0)

In [ ]:
a1.shape,a1l.shape, a2.shape, a2l.shape, a3.shape, a3l.shape

((1200, 2), (2400,), (1200,), (0,), (0,), (0,))

In [ ]:
# ar=(arr1[0][1]-amin)/(amax-amin)

In [ ]:
np.array(a2[0]).shape

(97, 1, 1000)

To use Deep Learning or any ML model wE need fixed size input.</br> We check feature shapes and get min and max feature shapes.</br>
Below different feature shapes are recorded.

In [ ]:
#different shapes of video feature
vfs=[]#vid_feat_shape
for vf in a1:
    if np.array(vf[0]).shape not in vfs:
        vfs.append(np.array(vf[0]).shape)
for vf in a2:
    if np.array(vf).shape not in vfs:
        vfs.append(np.array(vf).shape)

mfs=[]#mfcc_feat_shape
for mf in a1:
    if np.array(mf[1]).shape not in mfs:
        mfs.append(np.array(mf[1]).shape)
for mf in a3:
    if mf.shape not in mfs:
        mfs.append(np.array(mf).shape)

min and max feature shapes for video and audio are obtained.

In [ ]:
imax=0
imin=10000
for i in vfs:
    if i[0]>imax:
        imax=i[0]
    if i[0]<imin:
        imin=i[0]
jmax,jmin=0,10000
for i in mfs:
    if i[1]>jmax:
        jmax=i[1]
    if i[1]<jmin:
        jmin=i[1]

In [ ]:
imin,imax,jmin,jmax

(88, 157, 127, 228)

Making all feature video and audio respectively to same size

In [ ]:
#np.array(a1[0][0]).reshape(len(a1[0][0]))
#def video_frame_bal(a1)
a1_1=[]
for i in a1:
    arr=np.array(i[0])
    arr=arr.reshape(arr.shape[0],arr.shape[2])
    diff=imax-arr.shape[0]
    if diff%2==1:
        front=np.zeros((diff//2,1000),dtype='float')
        end=np.zeros(((diff//2)+1,1000),dtype='float')
    else:
        front=np.zeros((diff//2,1000),dtype='float')
        end=np.zeros((diff//2,1000),dtype='float')
    a1_1.append(np.concatenate((front,arr,end), axis=0))
a1_1=np.array(a1_1)

In [ ]:
#def audio_frame_bal(a1):
jmax=250
a1_2=[]
for i in a1:
    arr=i[1]
    diff=jmax-arr.shape[1]
    if diff%2==1:
        front=np.zeros((44,diff//2),dtype='float')
        end=np.zeros((44,(diff//2)+1),dtype='float')
    else:
        front=np.zeros((44,diff//2),dtype='float')
        end=np.zeros((44,diff//2),dtype='float')
    a1_2.append(np.concatenate((front,arr,end), axis=1))
a1_2= np.array(a1_2)

In [ ]:
a2_1=[]
for i in a2:
    arr=np.array(i)
    arr=arr.reshape(arr.shape[0],arr.shape[2])
    diff=imax-arr.shape[0]
    if diff%2==1:
        front=np.zeros((diff//2,1000),dtype='float')
        end=np.zeros(((diff//2)+1,1000),dtype='float')
    else:
        front=np.zeros((diff//2,1000),dtype='float')
        end=np.zeros((diff//2,1000),dtype='float')
    a2_1.append(np.concatenate((front,arr,end), axis=0))
a2_1=np.array(a2_1)

In [ ]:
# making audio with zero array for the sample with only video. Now all samples have audio and video.
a2_2=np.zeros((1200,44,250), dtype=float)

In [ ]:
a1_1.shape,a1_2.shape, a2_1.shape, a2_2.shape

((1200, 157, 1000), (1200, 44, 250), (1200, 157, 1000), (1200, 44, 250))

Normalize audio feature

In [ ]:
a1_2= (a1_2-amin)/(amax-amin)
a2_2= (a2_2-amin)/(amax-amin)

Reshape Audio feature

In [ ]:
a1_2= a1_2.reshape(a1_2.shape[0],-1,1000)
a2_2= a2_2.reshape(a2_2.shape[0],-1,1000)

In [ ]:
a1_1.shape,a1_2.shape, a2_1.shape, a2_2.shape

((1200, 157, 1000), (1200, 11, 1000), (1200, 157, 1000), (1200, 11, 1000))

In [ ]:
A11=np.concatenate((a1_1,a1_2), axis=1)
A22=np.concatenate((a2_1,a2_2), axis=1)

In [ ]:
A11.shape,A22.shape

((1200, 168, 1000), (1200, 168, 1000))

Each sample now is of uniform shape. and contain audio and video feature. 
Finally we have data to train our multimodal algorithm.

In [ ]:
data= np.concatenate((A11,A22),axis=0)

In [ ]:
label= np.concatenate((label1,a21l,a31l,label2,a22l,a32l),axis=0)

In [ ]:
data.shape,label.shape

((2400, 168, 1000), (2400,))

Just free_ing space ram is full.

In [ ]:
A11,A22, a1, a2,a3,a1l,a2l,a3l=0,0,0,0,0,0,0,0
arr1,arr2,arr3,label1,label2,label3=0,0,0,0,0,0
a21,a22,a23,a21l,a22l,a23l=0,0,0,0,0,0
a31,a32,a33,a31l,a32l,a33l=0,0,0,0,0,0
a1_1,a1_2, a2_1, a2_2= 0,0,0,0

To create an image type input with three channel.</br>
168507/3= 56169</br>
237*237 = 56169

In [ ]:
data= data.reshape(len(data),-1)
data=np.concatenate((data,np.zeros((len(data),507), dtype=float)),axis=1)
data= data.reshape(len(data), 237,237,3)

label from numral to categorical

In [ ]:
label2=[]
for i in label:
    label2.append(i-1)
label= np.array(label2,dtype=int)
label=to_categorical(label)

(OPTIONAL)</br>
Trying to use normal resnet nput shape

In [ ]:
data2=[]
for i in data:
    data2.append(cv2.resize(i,(224,224)))
data2=np.array(data2)
data2.shape

(2400, 224, 224, 3)

In [ ]:
data.shape, label.shape

((2400, 237, 237, 3), (2400, 8))

In [ ]:
# base_model.layers.pop(0)

Model

In [ ]:
input = Input(shape=(224,224, 3), name='image_input')
for layer in base_model.layers:
    layer.trainable = True
x = Flatten()(base_model(input))
x = Dense(200, activation='relu')(x)
#x = Dense(100, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(8, activation='softmax')(x)
model = Model(inputs=input, outputs=x)
opt = SGD(learning_rate=1e-4)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(data2,label, validation_split=0.15,epochs=50)#, batch_size=200)

The dataset till actor 20 don't have only audio samples.

In [ ]:
#def audio_frame_bal(a1):
a3_1=[]
for i in a3:
    arr=i
    diff=jmax-arr.shape[1]
    if diff%2==1:
        front=np.zeros((44,diff//2),dtype='float')
        end=np.zeros((44,(diff//2)+1),dtype='float')
    else:
        front=np.zeros((44,diff//2),dtype='float')
        end=np.zeros((44,diff//2),dtype='float')
    a3_1.append(np.concatenate((front,arr,end), axis=1))
a3_1= np.array(a3_1)

In [ ]:
a3_1.shape

(0,)

In [ ]:
# #files.download('1.mp3')
# Audio('1.mp3')

In [ ]:
# doc_name='02-01-02-02-02-02-01.mp4'
# n=doc_name.split('.mp4')
# w=int(n[0].split('-')[0])#01 = full-AV, 02 = video-only, 03 = audio-only
# l=int(n[0].split('-')[2])#01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised
# if (w==1):#audio video both
#     # nn=str(vid[0])
#     # vid_name="/normal_final/"+nn
#     arr=[]
#     video = cv2.VideoCapture(doc_name)
#     while(True): 
#         ret, frame = video.read() 
#         if ret == True:  
#             #input=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
#             frame=cv2.resize(frame,(224,224))
#             feat=frame.reshape(1,224,224,3)
#             arr.append(base_model.predict(feat))
#         else:
#             print("done",vid[0])
#             video.release()
#             cv2.destroyAllWindows()
#             break
    
#     subprocess.call(['ffmpeg', '-i', 'Actor_01/'+l[3], '1.mp3'])
#     x,sr=librosa.load('1.mp3')
#     x2=librosa.to_mono(x)
#     mf.append(librosa.feature.mfcc(x2, sr=22050, n_mfcc=44))
        
# elif (w==2):
#     #only video
#     arr=[]
#     video = cv2.VideoCapture(doc_name)
#     while(True): 
#         ret, frame = video.read() 
#         if ret == True:  
#             #input=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
#             frame=cv2.resize(frame,(224,224))
#             feat=frame.reshape(1,224,224,3)
#             arr.append(base_model.predict(feat))
#         else:
#             print("done",vid[0])
#             video.release()
#             cv2.destroyAllWindows()
#             break
# else:
#     #only audio
#     subprocess.call(['ffmpeg', '-i', 'Actor_01/'+l[3], '1.mp3'])
#     x,sr=librosa.load('1.mp3')
#     x2=librosa.to_mono(x)
#     mf.append(librosa.feature.mfcc(x2, sr=22050, n_mfcc=44))


2 2


In [ ]:
# files.upload()